In [5]:
import requests, json
import pandas as pd
import time
 
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json


In [6]:
check = get_location('서울대입구역 2호선')

In [27]:
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
    }
    response = requests.get(url, headers=headers, params={'query': address}).json()

    long = response['documents'][0]['x'] 
    lati = response['documents'][0]['y']
    return long, lati


In [28]:
long, lati = get_location('서울대입구역')

In [29]:
long, lati

('126.952713197762', '37.4812845080678')

In [35]:
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
headers = {
"Authorization": f"KakaoAK {REST_API_KEY}"
}
response = requests.get(url, headers=headers, params={'query': '강남 MICE 관광특구'}).json()


In [36]:
response

{'documents': [],
 'meta': {'is_end': True,
  'pageable_count': 0,
  'same_name': {'keyword': 'MICE 관광특구',
   'region': [],
   'selected_region': '서울 강남구'},
  'total_count': 0}}

In [ ]:
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
    }
    response = requests.get(url, headers=headers, params={'query': address}).json()

    long = response['documents'][0]['x'] 
    lati = response['documents'][0]['y']
    return long, lati


# 지도 거리 계산

In [30]:
import numpy as np


In [57]:
def haversine(lat1, lon1, lat2, lon2):
    # 모든 위도 경도는 라디안 단위로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # 위도와 경도 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 허버사인 공식 사용
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # 6,371km는 지구의 반지름
    km = 6371 * c
    return km


In [32]:
# 예제 데이터프레임 생성
data = {'이름': ['장소A', '장소B', '장소C'],
        '위도': [37.5665, 35.1796, 33.4507],
        '경도': [126.9780, 129.0756, 126.5707]}
df = pd.DataFrame(data)

# 기준 위치 설정 (예: 서울 중심부)
base_lat, base_lon = 37.5665, 126.9780

# 각 위치와의 거리 계산
df['거리'] = df.apply(lambda row: haversine(base_lat, base_lon, row['위도'], row['경도']), axis=1)


In [33]:
closest_location = df.loc[df['거리'].idxmin()]
print("가장 가까운 위치:", closest_location['이름'], "거리:", closest_location['거리'], "km")


가장 가까운 위치: 장소A 거리: 0.0 km


In [34]:
closest_location

이름        장소A
위도    37.5665
경도    126.978
거리        0.0
Name: 0, dtype: object

## 실제 데이터 넣어보기

In [50]:
# 키워드로 위도 경도 반환받기
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
    }
    response = requests.get(url, headers=headers, params={'query': address}).json()

    long = response['documents'][0]['x'] 
    lati = response['documents'][0]['y']
    return float(lati), float(long)


In [51]:
long, lati = get_location('서울대입구역')

In [54]:
df = pd.read_csv('./data/landmark_final.csv')

In [70]:
df['거리'] = df.apply(lambda row: haversine(float(lati), float(long), row['위도'], row['경도']), axis=1)
closest_location = df.loc[df['거리'].idxmin()]['랜드마크']


In [71]:
closest_location

'서울대입구역'

In [73]:
url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/citydata/1/2/{closest_location}"
response = requests.get(url)
data = json.loads(response.text)

In [75]:
data['CITYDATA']['LIVE_PPLTN_STTS'][0]

{'AREA_CONGEST_LVL': '보통',
 'AREA_CONGEST_MSG': '사람이 몰려있을 수 있지만 크게 붐비지는 않아요. 도보 이동에 큰 제약이 없어요.',
 'AREA_PPLTN_MIN': '20000',
 'AREA_PPLTN_MAX': '22000',
 'MALE_PPLTN_RATE': '49.5',
 'FEMALE_PPLTN_RATE': '50.5',
 'PPLTN_RATE_0': '1.4',
 'PPLTN_RATE_10': '7.1',
 'PPLTN_RATE_20': '37.0',
 'PPLTN_RATE_30': '20.1',
 'PPLTN_RATE_40': '11.8',
 'PPLTN_RATE_50': '10.6',
 'PPLTN_RATE_60': '7.1',
 'PPLTN_RATE_70': '5.0',
 'RESNT_PPLTN_RATE': '33.4',
 'NON_RESNT_PPLTN_RATE': '66.6',
 'REPLACE_YN': 'N',
 'PPLTN_TIME': '2024-04-25 20:00',
 'FCST_YN': 'Y',
 'FCST_PPLTN': [{'FCST_TIME': '2024-04-25 21:00',
   'FCST_CONGEST_LVL': '붐빔',
   'FCST_PPLTN_MIN': '20000',
   'FCST_PPLTN_MAX': '22000'},
  {'FCST_TIME': '2024-04-25 22:00',
   'FCST_CONGEST_LVL': '붐빔',
   'FCST_PPLTN_MIN': '20000',
   'FCST_PPLTN_MAX': '22000'},
  {'FCST_TIME': '2024-04-25 23:00',
   'FCST_CONGEST_LVL': '약간 붐빔',
   'FCST_PPLTN_MIN': '20000',
   'FCST_PPLTN_MAX': '22000'},
  {'FCST_TIME': '2024-04-26 00:00',
   'FCST_CONGEST_LVL'

In [77]:
# 1시간 뒤
data['CITYDATA']['LIVE_PPLTN_STTS'][0]['FCST_PPLTN'][0]

{'FCST_TIME': '2024-04-25 21:00',
 'FCST_CONGEST_LVL': '붐빔',
 'FCST_PPLTN_MIN': '20000',
 'FCST_PPLTN_MAX': '22000'}

In [78]:
# 12시간 뒤까지 가능
data['CITYDATA']['LIVE_PPLTN_STTS'][0]['FCST_PPLTN'][-1]

{'FCST_TIME': '2024-04-26 08:00',
 'FCST_CONGEST_LVL': '약간 붐빔',
 'FCST_PPLTN_MIN': '20000',
 'FCST_PPLTN_MAX': '22000'}

In [79]:
len(data['CITYDATA']['LIVE_PPLTN_STTS'][0]['FCST_PPLTN'])

12

In [80]:
# 혼잡지수
data['CITYDATA']['LIVE_PPLTN_STTS'][0]['AREA_CONGEST_LVL']

'보통'

In [81]:
# 혼잡메시지
data['CITYDATA']['LIVE_PPLTN_STTS'][0]['AREA_CONGEST_MSG']

'사람이 몰려있을 수 있지만 크게 붐비지는 않아요. 도보 이동에 큰 제약이 없어요.'